In [ ]:
# Uncomment this if you are using Colab
# ## Mount google drive: If your dataset is saved on google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


# Step 1: Load the images

# b) Load the first image
imFirst = np.array(Image.open('。/faces/s1/1.pgm'))
imFirst = imFirst.astype('float32')
imFirst /= 255.0
imFirst = np.clip(imFirst, 0.0, 1.0)
plt.imshow(imFirst, cmap='gray')
plt.show()

# c) Load the training images (40 persons X 7 images)
nTrainImages = 40 * 7  # number of training images
height, width = imFirst.shape  # The images have the same size. Use the first one to calculate the number of pixels

imagesTrain = np.zeros((height * width, nTrainImages))  # Matrix with the training images as rows
# TODO: write a piece of code to load all training data into imagesTrain
# NOTE: You can use another more 'python' way to replace this c++ style data assignment: append every image in a list, and use np.stack to stack them into a batch, then flatten only width and height dimension

# d) read testing images.
nTestImages = 40 * 3  # testing images
imagesTest = np.zeros((height * width, nTestImages))
# %%%TODO: write a piece of code to load all test data into imagesTest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 2: PCA
# a) find the mean image
mean_face = np.mean(imagesTrain, axis=1)
# TODO: visualize mean_face


# b) mean-shifted input images
shifted_images = imagesTrain - mean_face[:, np.newaxis]

# This is a pre-step to optimize calculation. (see your lecture notes)
# TODO: normally we should find the covariance of shifted_images but it will be a 10304*10304 matrix!
# to be optimized and avoid "out of memory" error, we use a trick!
# Compute Y'*Y 
YY = 1/shifted_images.shape[0] * np.dot(shifted_images.T, shifted_images)

# c) Compute eigenvectors
evalues, evectors = np.linalg.eig(YY)
evectors = np.dot(shifted_images, evectors)

# d) Sort eigenvectors based on their corresponding eigenvalues
# TODO: put the results in "evectors" variable

# e) only retain the top 'num_eigenfaces' eigenvectors (i.e. the principal components)
num_eigenfaces = 30
# TODO: put the results in "evectors" variable

# Normalize the eigenvectors so || evector_i|| = 1
for i in range(num_eigenfaces):
    evectors[:, i] /= np.linalg.norm(evectors[:, i])

# f) project the images into the subspace to generate the feature vectors
# TODO: put the results in "features" variable


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Step 3: Analysis of eigen faces
# a) display the eigenvectors
fig, axes = plt.subplots(1, num_eigenfaces, figsize=(12, 4))
for n in range(num_eigenfaces):
    evector = evectors[:, n].reshape(height, width)
    axes[n].imshow(evector, cmap='gray')
    axes[n].axis('off')
plt.show()

In [ ]:
# b) display the cumulative eigenvalues
# TODO: hint: "cumsum" function for accumulation

In [ ]:
# Step 4: Identify a person
testIm = 100  # chose a person index
input_image = np.expand_dims(imagesTest[:, testIm],-1) 
# Tips: a shortcut way in python is input_image = imagesTest[:, testIm][..., None]

shifted_input_image = input_image - mean_face[:, np.newaxis]
shifted_imagesTrain = imagesTrain - mean_face[:, np.newaxis]

# calculate the similarity of the input to each training image
# TODO: put the result in "similarity_score" variable

# find the image with the highest similarity
match_ix = np.argmax(similarity_score)

# display the result
fig, ax = plt.subplots(1, 2)
ax[0].imshow(input_image.reshape(height, width), cmap='gray')
ax[0].axis('off')
ax[1].imshow(imagesTrain[:, match_ix].reshape(height, width), cmap='gray')
ax[1].axis('off')
ax[1].set_title(f"matches to the {match_ix}th training sample with score {similarity_score[match_ix]}")
plt.show()

In [ ]:
# Step 5: Cluster the persons

# a) Cluster the original images
# TODO: put the results in "clusters" variable. "clusters" indicates cluster number for each sample. 
#       please refer to KMeans in popular machine learning library: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

# Show the the images that were assigned to cluster 1
img_index = np.where(clusters == 0)[0]

for i in range(min(len(img_index), 7)):
    plt.figure(111)
    plt.subplot(1, 7, i+1)
    img = np.reshape(imagesTrain[:, img_index[i]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
# b) Now cluster on the reduced space
# TODO: put the results in "clusters_pca" variable.


# Show the the images that were assigned to cluster 1
img_index = np.where(clusters_pca == 0)[0]
for i in range(min(len(img_index), 7)):
    plt.figure(111)
    plt.subplot(1, 7, i+1)
    img = np.reshape(imagesTrain[:, img_index[i]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

In [ ]:
# c) Try agglomerative clustering on some images
# TODO: hint: use linkage command for agglomerative clustering with "single" metric and put the results in "Z_agglo" variable
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

checkImages = list(range(8,21)) # You can use any index, here index 8-20 have been selected

Z_agglo = linkage(imagesTrain.T[checkImages], method='single')

fig = plt.figure()
dn = dendrogram(Z_agglo)
c = fcluster(Z_agglo, t=2, criterion='maxclust') # Chage the value of t to change the number of clusters.

# Show the the images that were assigned to cluster 1
img_index = np.where(c == 1)[0] # change this value to see image associated with different clusters.
for i in range(min(len(img_index), 5)):
    plt.figure(111)
    plt.subplot(1, 5, i+1)
    img = np.reshape(imagesTrain[:, checkImages[img_index[i]]], (height, width))
    plt.imshow(img, cmap='gray')
    plt.axis('off')